# MuMoT Demonstration Notebook: SEIR-Derived Epidemiological Models of COVID-19 and Social Distancing<a class="tocSkip">

## Multiscale Modelling Tool <a class="tocSkip">

*James A. R. Marshall, Department of Computer Science, University of Sheffield*

# Introduction
    
This notebook aims to help explain the possible epidemiology of and effects of different public interventions on the COVID-19 novel coronavirus infectious disease, caused by the SARS-CoV-2 virus (WHO, <a href='#references'>2020</a>). Models are presented to show how different health policies may impact on the progression of the disease through a population. <i>The models are simple 'toy' models for illustrative purposes, although model parameters are taken from the literature on the disease</i>.

_**If you are a non-specialist, you may simply want to go to the `Kernel` menu above, select `Restart & Run All`, then skip to the interactive plots and use these, along with reading their accompanying bold text.**_

In [ ]:
import mumot

# Model Definitions

Our model is adapted from the standard SEIR model (Susceptiple, Exposed, Infectious, Recovered). The SEIR model is a model used to represent infectious diseases with a latent period following infection, before an individual becomes infectious, and has been used in modelling COVID-19 (<i>e.g.</i>Wu <i>et al.</i>, <a href='#references'>2020</a>). However, since COVID-19 may exhibit significant presymptomatic infectiousness (Anderson <i>et al.</i>, <a href='#references'>2020</a>; Du <i>et al.</i><a href='#references'>2020</a>, <i>Tindal et al.</i>, <a href='#references'>2020</a>), we also include an infection process from members of the $E$ class, which is not usual in SEIR models; comparatively few models of COVID-19 to date have explicitly included infection by asymptomatic individuals (see Cao <i>et al.</i> (<a href='#references'>2020</a>) and Li <i>et al.</i>(<a href='#references'>2020</a>) for exceptions). Since asymptomatic individuals apparently can also recover directly without becoming symptomatic, we also adapt the SEIR model to allow direct recovery from the exposed class $E$. For simplicity, we assume the same infection rate $r$ regardless of whether a person is in $E$ or $I$.

For COVID-19 infection rates are assumed to be higher than recovery rates.

For health systems, the key population to track is the infectious population $I$, which in our model represents the symptomatic members of the population, since these are the ones that will require healthcare resources such as medicines, medical callouts, or hospitalisation. _Note that this model does not explicitly represent deaths from infection - all population members eventually recover, which is clearly not the case for COVID-19_

Thus our model is defined as follows:

In [ ]:
%%model
$
S + I -> I + E: r * s
S + E -> E + E: r * (1 - s)
E -> I: a
E -> R: g
I -> R: g
$

In [ ]:
fullmodel = mumot.parseModel(In[-2])
finitemodel = fullmodel.substitute('S = N - I - E - R')
finitemodel.show()

# Results
In the following, we analyse the models using both infinite-population ODEs, and finite population spatial stochastic simulations. _The latter are particularly good for allowing us to vary *social distancing* and see the effects on the infection's progress_.

## Social Distancing Across the Population
We parameterise the model to approximate the characteristics of COVID-19; the purpose of this demonstration is to show the effects of social distancing, so exact parameterisation is not required but could be found from the literature (see sample literature in <a href='#references'>References</a>). Here we choose $r=2.5$ and $a=g=1/2$; estimates of infectiousness vary but are higher than previous comparable diseases (see Liu <i>et al.</i>, <a href='#references'>2020</a>, for a review) estimates of asymptomatic transmissions share of total infections vary between being a minority (Du <i>et al.</i>, <a href='#references'>2020</a>) and, potentially, a majority (Tindal <i>et al.</i>, <a href='#references'>2020</a>; Li <i>et al.</i>, <a href='#references'>2020</a>); we assume infectiousness for members of $E$ is half that of members in $I$ ($s=2/3$) (Li <i>et al.</i>, <a href='#references'>2020</a>). We then present a controller in which there is a single slider to change the degree of social isolation. This enables us to easily compare the effects of no social isolation (a 'well-mixed' population, left) with social isolation (a spatial, partially connected population, right).

_**You can see how social distancing affects the spread of the disease within the population by looking inside the population as individuals mingle and interact. Open the 'Advanced options' tab and vary the interaction range between 0 (full social distancing) and 1 (no social distancing), then observe the effect on the population. Recall that green individuals are the ones who may need hospital treatment, and the red individuals are recovered. Implementing social distancing means the virus has fewer opportunities to jump between individuals, so doesn't spread as fast.**_

In [ ]:
agentcont1 = finitemodel.multiagent(ylab = 'cases', initialState = {'E': 0.05, 'I': 0.0, 'R': 0.0, 'S': 0.95}, initWidgets={'netParam':[1.0, 0, 1.0, 0.05]}, maxTime = 10.0, timestepSize = 0.25, randomSeed = 731529356, netType = 'dynamic', motionCorrelatedness = 0.5, particleSpeed = 0.01, showTrace = False, showInteractions = False, visualisationType = 'graph', plotProportions = True, realtimePlot = True, runs = 1, aggregateResults = True, bookmark = False, silent = False, params = [('s', 2/3),('a', 0.5), ('g', 0.5), ('r', 2.0), ('plotLimits', 1), ('systemSize', 100.0)])

Open the 'Advanced options' tab again and vary the interaction range between 0 (full social distancing) and 1 (no social distancing), then observe the effect on a sample infection trajectory in the right hand plot, compared to the left hand reference plot (no social distancing). The plot shows the fractions of the total population that are in the different disease states, indicated by the figure lengends. <i>Note that social distancing not only reduces the peak number of cases needing medical attention, but also the total number of infections.</i>

_**As you vary the interaction range slider between full social contact (1.0) and no social contact (0.0) you should observe a threshold effect at around 0.10, where the progress of the disease is drastically reduced; the maximum proportion of cases at any point in time (green line) reduces.**_

In [ ]:
multicont1 = mumot.MuMoTmultiController(
    [finitemodel.integrate(showStateVars=['E', 'I', 'R'], ylab = 'cases', initialState = {'E': 0.05, 'I': 0.0, 'R': 0.0, 'S': 0.95}, maxTime = 10.0, plotProportions = True, silent = True, conserved = True, bookmark = False),
     finitemodel.multiagent(ylab = 'cases', initialState = {'E': 0.05, 'I': 0.0, 'R': 0.0, 'S': 0.95}, initWidgets={'netParam':[1.0, 0, 1.0, 0.05]}, maxTime = 10.0, timestepSize = 0.25, randomSeed = 731529356, netType = 'dynamic', motionCorrelatedness = 0.5, particleSpeed = 0.01, showTrace = False, showInteractions = False, visualisationType = 'evo', plotProportions = True, realtimePlot = False, runs = 1, aggregateResults = True, silent = True, bookmark = False)], params = [('s', 2/3),('a', 0.5), ('g', 0.5), ('r', 2.5), ('plotLimits', 1), ('systemSize', 100.0)],
     choose_yrange = [0, 1])

## Social Distancing vs Isolating Symptomatic Cases

We now adapt our model to examine the possible relative effectiveness of social distancing, compared to isolating only symptomatic individuals. The basic infectiousness of the disease $r$ can be varied, as can the share of infections generated by symptomatic individuals $I$ ($s$). We make a new model in which symptomatic individuals are isolated so reduce their infectivity ten-fold, and introduce a social distancing parameter $d$ that scales contacts between asymptomatic individuals ($E$) and susceptible individuals ($S$). For this scenario we now assume asymptomatic infectiousness is twice that of symptomatic (<i>cf.</i> Tindal <i>et al.</i>, <a href="#references">2020</a>; Li <i>et al.</i>, <a href="#references">2020</a>), $s=1/3$.

In [ ]:
%%model
$
S + I -> I + E: r * s / 10
S + E -> E + E: r * d * (1 - s)
E -> I: a
E -> R: g
I -> R: g
$

_**Vary the social distancing parameter $d$ to see the difference that social distancing by asymptotic carriers makes (righthand plot), on top of isolation only of symptomatic cases (lefthand plot). You can also vary the infectiousness of the disease ($r$) and proportion of infections due to symptomatic carriers ($s$), since there is some uncertainty around these.**_

In [ ]:
compmodel1 = mumot.parseModel(In[-2]).substitute('S = N - I - E - R')
compmodel1.show()

In [ ]:
multicont2 = mumot.MuMoTmultiController(
    [compmodel1.integrate(showStateVars=['E', 'I', 'R'], ylab = 'cases', initialState = {'E': 0.05, 'I': 0.0, 'R': 0.0, 'S': 0.95}, maxTime = 10.0, choose_yrange = [0, 1], plotProportions = True, silent = True, conserved = True, bookmark = False, params = [('d', 1)]),
     compmodel1.integrate(showStateVars=['E', 'I', 'R'], ylab = 'cases', initialState = {'E': 0.05, 'I': 0.0, 'R': 0.0, 'S': 0.95}, maxTime = 10.0, choose_yrange = [0, 1], plotProportions = True, silent = True, conserved = True, bookmark = False)],
     initWidgets={'r':[2.5, 0, 4, 0.1],'s':[0.35, 0, 1, 0.05], 'd':[1, 0.5, 1, 0.05]}, params = [('a', 0.5), ('g', 0.5), ('plotLimits', 1), ('systemSize', 10)])

# Whole-Population Social Distancing vs Distancing for High Risk Groups

Similarly to examining the effects of isolation only on symptomatic individuals above, Benjamin Kerr (<a href="#references">2020</a>) and Omar Cornejo (<a href="#references">2020</a>) constructed a model of social distancing for different risk groups. Here we adapt our modified SEIR model to include their concept of high and low risk groups. The basic rate parameters for the disease are the same for each, but social distancing factors can be applied to each independently.

In [ ]:
%%model
$
S_h + E_h -> E_h + E_h: r * (1 - s) * d_h
S_l + E_h -> E_l + E_h: r * (1 - s) * d_l
S_h + E_l -> E_h + E_l: r * (1 - s) * d_h
S_l + E_l -> E_l + E_l: r * (1 - s) * d_l
S_h + I_h -> E_h + I_h: r * s * d_h
S_l + I_h -> E_l + I_h: r * s * d_l
S_h + I_l -> E_h + I_l: r * s * d_h
S_l + I_l -> E_l + I_l: r * s * d_l
E_l -> I_l: a
E_h -> I_h: a
E_l -> R: g
E_h -> R: g
I_l -> R: g
I_h -> R: g
$

In [ ]:
compmodel2 = mumot.parseModel(In[-2]).substitute('S = N - I_h - I_l - E_h - E_l - R')
compmodel2.show()

The model is paramaterised as follows: The susceptible population is 85:15 low risk:high risk, and initial carriers are equally divided between the low risk and high risk groups. The initial 5% of infected individuals are half low risk, half high risk. As above, infection rates of symptomatic ($r$) and asymptomatic ($r * s$) can be manipulated.

_**In the following plot you can manipulate the extent of social distancing for the high-risk ($d_h$) and low-risk ($d_l$) groups, from 1 (no social distancing for that group) to 0 (full social isolation for that group). The plots show (top-left) disease progression with no social distancing measures on any group; (top-right) progression with social distancing only on the high-risk group (bottom-left) progression with social distancing only on the low-risk group; (bottom-right) progression with social distancing only on the high-risk group. Low risk (dashed cyan) and high risk (solid red) infected populations are shown**_

In [ ]:
multicont2 = mumot.MuMoTmultiController(
    [compmodel2.integrate(showStateVars=['I_h', 'I_l'], ylab = 'cases', initialState = {'E_h': 0.025, 'I_h': 0.0, 'E_l': 0.025, 'I_l': 0.0, 'R': 0.0, 'S_h': 0.15, 'S_l': 0.8}, maxTime = 10.0, choose_yrange = [0, 0.3], plotProportions = True, silent = True, conserved = True, bookmark = False, params = [('d_h', 1), ('d_l', 1)]),
     compmodel2.integrate(showStateVars=['I_h', 'I_l'], ylab = 'cases', initialState = {'E_h': 0.025, 'I_h': 0.0, 'E_l': 0.025, 'I_l': 0.0, 'R': 0.0, 'S_h': 0.15, 'S_l': 0.8}, maxTime = 10.0, choose_yrange = [0, 0.3], plotProportions = True, silent = True, conserved = True, bookmark = False, params = [('d_l', 1)]),
     compmodel2.integrate(showStateVars=['I_h', 'I_l'], ylab = 'cases', initialState = {'E_h': 0.025, 'I_h': 0.0, 'E_l': 0.025, 'I_l': 0.0, 'R': 0.0, 'S_h': 0.15, 'S_l': 0.8}, maxTime = 10.0, choose_yrange = [0, 0.3], plotProportions = True, silent = True, conserved = True, bookmark = False, params = [('d_h', 1)]),
     compmodel2.integrate(showStateVars=['I_h', 'I_l'], ylab = 'cases', initialState = {'E_h': 0.025, 'I_h': 0.0, 'E_l': 0.025, 'I_l': 0.0, 'R': 0.0, 'S_h': 0.15, 'S_l': 0.8}, maxTime = 10.0, choose_yrange = [0, 0.3], plotProportions = True, silent = True, conserved = True, bookmark = False)],
     initWidgets={'r':[2.5, 0, 4, 0.1],'s':[0.35, 0, 1, 0.05], 'd_{h}':[0.25, 0, 1, 0.05], 'd_{l}':[0.25, 0, 1, 0.05]}, params = [('a', 0.5), ('g', 0.5), ('plotLimits', 1), ('systemSize', 10)])

# Discussion
_**The purpose of this notebook has been to highlight, using simple models that still capture key aspects of the disease, how various public health policy interventions might affect
the spread of COVID-19. The key messages are:**_
<ul>
<li>social distancing works by reducing the opportunities for the virus to jump between hosts.
<li>if asymptomatic cases are relatively infectious, isolating symptomatic individuals alone is likely to be comparatively ineffective compared to social distancing measures in the wider population.
<li>social distancing on the much larger low risk group is much more effective at reducing peak infections that social distancing on the high risk group alone. 
</ul>


# References 
<a id='references'></a>
<ul>
<li>Anderson, R. M., Heesterbreek, H., Klinkenberg, D. and Hollingsworth, T. D. (2020) How will country-based mitigation measures influence the course of the COVID-19 epidemic? <i>The Lancet</i> (online first). <a href="https://doi.org/10.1016/S0140-6736(20)30567-5">doi:10.1016/S0140-6736(20)30567-5</a>.
<li>Cao, Z., Zhang, Q., Liu, X., Pfeiffer, D., Jia, Z., Song, H. and Zeng, D. D. (2020) Estimating the effective reproduction number of the 2019-nCoV in China. <i>medRxiv</i> <a href="https://doi.org/10.1101/2020.01.27.20018952">doi:10.1101/2020.01.27.20018952</a>.
<li>Cornejo, O. E. (2020) Description of SEIR-MODEL to try to understand the SARS-CoV-2 epidemic <a href="https://github.com/oeco28/SEIR-MODEL">https://github.com/oeco28/SEIR-MODEL</a> [<i>accessed March 17th 2020</i>]
<li>Du, Z. Xu, X., Wu, Y., Wang, L., Cowling, B. J., Meyers, L. A. (2020) The serial interval of COVID-19 from publicly reported confirmed cases. <i>medRxiv</i> <a href="https://doi.org/10.1101/2020.02.19.20025452">doi:10.1101/2020.02.19.20025452</a>.
<li>Kerr, B. (2020) SEIR Risk Model <a href="https://github.com/evokerr/SEIR_Risk_Model">https://github.com/evokerr/SEIR_Risk_Model</a> [<i>accessed March 17th 2020</i>]
<li>Li, R., Pei, S., Chen, B., Song, Y., Zhang, T., Yang, W., Shaman, J. (2000) Substantial undocumented infection facilitates the rapid dissemination of novel coronavirus (SARS-CoV2). <i>Science</i> <a href="https://doi.org/10.1126/science.abb3221">doi:10.1126/science.abb3221</a>
<li>Liu, Y., Gayle, A. A., Wilder-Smith, A., Rocklov, J. (2020) The reproductive number of COVID-19 is higher compared to SARS coronavirus. <i>Journal of Travel Medicine</i> <b>27</b> <a href="https://doi.org/10.1093/jtm/taaa021">doi:10.1093/jtm/taaa021</a>
<li>Tindal, L. C., Coombe, M., Stockdale, J. E., Garlock, E. S., Lau, W. Y. V., Saraswat, M., Lee, Y.-H. B., Zhang, L., Chen, D. Wallinga, J. and Colijn, C. (2000) Transmission interval estimates suggest pre-symptomatic spread of COVID-19. <i>medRxiv</i> <a href="https://doi.org/10.1101/2020.03.03.20029983">doi:10.1101/2020.03.03.20029983</a>.
<li>World Health Organisation (2020) <a href="https://www.who.int/emergencies/diseases/novel-coronavirus-2019/technical-guidance/naming-the-coronavirus-disease-(covid-2019)-and-the-virus-that-causes-it">Naming the coronavirus disease (COVID-19) and the virus that causes it</a>. [<i>accessed March 16th 2020</i>].
<li>Wu, J. T., Leung, K. and Leugn, G. M. (2020) Nowcasting and forecasting the potential domestic and international spread of the 2019-nCoV outbreak originating in Wuhan, China: a modelling study. <i>The Lancet</i> <b>395</b>, 689-697. <a href="https://doi.org/10.1016/S0140-6736(20)30260-9">doi:10.1016/S0140-6736(20)30260-9</a>.
</ul>